# Protein display

## Setup

In [1]:
#| default_exp protein

In [2]:
#| export
from fasthtml.common import *
from fasthtml.jupyter import *

from fh_commons.core import *
from fh_commons.static import *

import json
import pandas as pd

In [32]:
#| export
def AF_display():
    "client side scripts"
    return Script(src="./imports/alphafold_display.js")

In [15]:
#| export
def pdbe_molstar():
    "Headers"
    s1 = Link(rel='stylesheet', type='text/css', href='https://cdn.jsdelivr.net/npm/pdbe-molstar@3.2.0/build/pdbe-molstar-light.css')
    s2 = Script(type='text/javascript', src='https://cdn.jsdelivr.net/npm/pdbe-molstar@3.2.0/build/pdbe-molstar-plugin.js')
    s3 = Style(
        '''
          #myViewer {
            float: left;
            width: 1000px;
            height: 550px;
            position: relative;
            margin: 20px;
          }
          .msp-plugin ::-webkit-scrollbar-thumb {
            background-color: #474748 !important;
            border-radius: 10px;
          }
          .msp-right-panel { 
            display: none !important; 
          }
          #protein-form {
            margin: 20px;
          }
          
        '''
    )
    return [s1, s2, s3]

In [16]:
hdrs = bootstrap_hdrs() + pdbe_molstar()

In [17]:
show(*hdrs)

In [7]:
from ngrok_token import *
url = start_ngrok(token)
url

'https://72c5-3-81-185-19.ngrok-free.app'

In [18]:
app,rt = fast_app(pico=False, live=True,debug=True,hdrs=hdrs)
server = JupyUvi(app)

In [14]:
# server.stop()

## Protein display

In [19]:
df = pd.read_parquet('data/AM.parquet')

In [30]:
@rt('/')
def get():
    form = Form(
        Input(type='text', name='protein-id', placeholder='Enter Protein ID (e.g., O15552)'),
        Button('Load Protein', type='button', id='load-protein-btn'),
        id='protein-form'
    )
    viewer = Div(id='myViewer')
    script = Script(src="./imports/alphafold_display.js")
    blank=Div(style='height: 600px;')

    return Titled('PDBe Mol* JS Plugin Demo - AlphaFold View', form, viewer, script, blank)


@rt('/api/protein/{uniprot_id}')
def get(uniprot_id: str):
    # Filter the dataframe for the given UniProt ID
    protein_data = df[df['uniprot'] == uniprot_id].copy()

    if protein_data.empty:
        return {'error': 'Uniprot id not found'}

    protein_data['residue'] = protein_data.site.str[1:].astype(int)

    # Convert the filtered data to a list of dictionaries
    site_data = protein_data[['residue', 'AM_pathogenicity']].to_dict('records')

    return {'site_data': site_data}

In [31]:
htmx(url,'/')

In [47]:
def get_spinner(id, cls='d-flex justify-content-center'):
    return Div(Div(
        Span('Loading...', cls='visually-hidden'),
        role='status',
        cls='htmx-indicator spinner-border',
        id=id,
    ),cls=cls)

In [49]:
%%s
get_spinner(id='sdf')

Example:

In [ ]:
@app.post
def example():
    ...
    send_select = Form(
            Hidden(value=df.to_json(),id='upload_df'),
            Div(select_list,button,cls='row g-2 align-items-center'),
            post=calculate,
            target_id='result',
            hx_indicator='#spinner',
        )
    
    spinner=get_spinner(id='spinner')
    
    return send_select, spinner, ...


## End

In [1]:
#| hide
import nbdev; nbdev.nbdev_export()